In [1]:
#@title # 1. Installs

from IPython.display import clear_output
%cd /content/
!git clone  https://github.com/cordob/ESRGAN-pytorch

clear_output()
print('Done!')

Done!


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c8e03c47-aa46-1d81-840c-5e47cac0b97d)


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
 !ls '/content/drive/MyDrive/Colab Notebooks/super resolution'

1  1.ipynb  2


In [10]:
!ls /content


drive  ESRGAN-pytorch  sample_data


In [11]:
%cd ESRGAN-pytorch

/content/ESRGAN-pytorch


 convert !!!

In [12]:
!python test.py --lr_dir '/content/drive/MyDrive/Colab Notebooks/super resolution/1' --sr_dir '/content/drive/MyDrive/Colab Notebooks/super resolution/2'

save seed2160atoon3.png


1024.pkl     avasdemon.pkl  ku2_teen.pkl  pretain.json
ava_met.pkl  avas_teen.pkl  kuvsh_v2.pkl  stylegan2-ffhq-config-f.pkl
avas_60.pkl  ffhq.pkl	    metfaces.pkl  teen.pkl


In [ ]:
!ls '/content/1024 pretained'

ls: cannot access '/content/1024 pretained': No such file or directory


In [ ]:
from google.colab import files
uploaded = files.upload()


pretained   모델 가져오기 

In [ ]:
!cp -r  '/content/drive/MyDrive/Colab Notebooks/1024 pretained/pretain.json' -r /content


In [ ]:
!ls /content

drive  pretain.json  sample_data  stylegan2-ada-pytorch


In [ ]:
#@title # 1.1 Neural Networks handling
%cd /content/
from IPython.display import clear_output
import json



with open('/content/pretain.json', 'r') as f:
    jnetw = json.loads(f.read())

nn_list = list(jnetw.keys())

network_name = '' #@#param {type:"string"}
download_all = True #@#param {type:"boolean"}

if download_all:
    for nn in nn_list:
        url = jnetw[nn]
        id =  url.split('/')[-2]
        network_f_name = nn + '.pkl'
        !gdown --id $id -O $network_f_name
    clear_output()
    network_name = 'ffhq'
    print('Network:', network_name)
else:
    if not network_name:
        print('List of available networks:')
        print(', '.join(nn_list))
    elif network_name not in nn_list:
        print('Error, You can not use {}! Only names from this list are available:'.format(network_name))
        print(', '.join(nn_list))
    else:
        clear_output()
        print('Network:', network_name)
        url = jnetw[network_name]
        id =  url.split('/')[-2]
        network_f_name = network_name + '.pkl'
        !gdown --id $id -O $network_f_name

clear_output()

print('All networks are downloded.')
print('List of available networks:')
print(', '.join(nn_list))

All networks are downloded.
List of available networks:
idol, black, asian, indian, nkuvshinov2, nkuvshinov1, avasdemon, ffhq, 10-19, 60+, ku2teen, metface, avasteen


In [ ]:
#@title # 1.2 Useful utility functions
%cd /content/stylegan2-ada-pytorch/
import os
import re
from typing import List, Optional

import click
import dnnlib
import numpy as np
import PIL.Image
import torch
from io import BytesIO
import legacy



import argparse
import numpy as np
import PIL.Image
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import os
import pickle
from google.colab import files

def generate_images(zs, truncation_psi):
    # Gs_kwargs = dnnlib.EasyDict()
    # Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    # Gs_kwargs.randomize_noise = False
    # if not isinstance(truncation_psi, list):
    #     truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    # for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
    #     Gs_kwargs.truncation_psi = truncation_psi[z_idx]
    #     noise_rnd = np.random.RandomState(1) # fix noise
    #     tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    #     images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
    #     imgs.append(PIL.Image.fromarray(images[0], 'RGB'))

    # for seed_idx, seed in enumerate(seeds):
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        # print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        # img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = G(z, None, truncation_psi=truncation_psi, noise_mode=noise_mode)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        imgs.append(PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB'))
        # PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        # z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True, save=False, id=None):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  #####
  if save:
      PIL.Image.fromarray(a).save(str(id[0]) + '.png', format)
  ####
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )





/content/stylegan2-ada-pytorch


In [ ]:
!ls /content

10-19.pkl      black.pkl   ku2teen.pkl	    sample_data
60+.pkl        drive	   metface.pkl	    stylegan2-ada-pytorch
asian.pkl      ffhq.pkl    nkuvshinov1.pkl
avasdemon.pkl  idol.pkl    nkuvshinov2.pkl
avasteen.pkl   indian.pkl  pretain.json


In [ ]:
!pwd

/content/stylegan2-ada-pytorch


In [ ]:
#@title # 2. Mix layers
# %cd /content/stylegan2-ada-pytorch/

from IPython.display import display, clear_output

size = 4

seeds = np.random.randint((2**32 - 1), size=size)


# seeds = [1964007513, 4236099628, 3156054576,
#         3335768848, 1628418723, 2439969215,
#         2076868819, 2569559432, 1161871982]

# seeds = [3700796578, 3157145896, 1066592900, 565298939]

# seeds = [771744419, 3456854398, 1121473508, 1489824126]


# seeds = [966377487, 1504967482, 661703051, 
#         3901841580]
#, 3825217503, 3560023859,
#         4210396737, 2035533728, 3148503014]

#############################################
 #@markdown onto
network_name1 = 'avasdemon' #@param {type: "string"}
 #@markdown from
network_name2 = 'nkuvshinov1' #@param {type: "string"}

noise_mode = 'const'
device = torch.device('cuda')
###
#########################



network_pkl0 = '/content/' + network_name2.strip() + '.pkl'

with dnnlib.util.open_url(network_pkl0) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

imgs0 = generate_images_from_seeds(seeds, 0.7)

#########################


network_pkl1 = '/content/' + network_name1.strip() + '.pkl'

network_pkl = '/content/stylegan2-ffhq-config-f.pkl'
with dnnlib.util.open_url(network_pkl1) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore


network_pkl2 = '/content/' + network_name2.strip() + '.pkl'

with dnnlib.util.open_url(network_pkl2) as f:
    G2 = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

##############################################


print(list(enumerate(list(seeds))), '\n')

size = len(seeds)

rows = 2
image_size = 0.4

size = len(seeds)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]



imgs1 = generate_images_from_seeds(seeds, 0.7)
# display(createImageGrid(imgs1, image_size , rows))



# G._modules['synthesis'].b4 = G2._modules['synthesis'].b4      #.__dir__()
# G._modules['synthesis'].b8 = G2._modules['synthesis'].b8      #.__dir__()
# G._modules['synthesis'].b16 = G2._modules['synthesis'].b16      #.__dir__()
# G._modules['synthesis'].b32 = G2._modules['synthesis'].b32      #.__dir__()
# G._modules['synthesis'].b64 = G2._modules['synthesis'].b64      #.__dir__()
# G._modules['synthesis'].b128 = G2._modules['synthesis'].b128      #.__dir__()
# G._modules['synthesis'].b256 = G2._modules['synthesis'].b256      #.__dir__()
# G._modules['synthesis'].b512 = G2._modules['synthesis'].b512      #.__dir__()
# G._modules['synthesis'].b1024 = G2._modules['synthesis'].b1024      #.__dir__()

#@markdown Blocks of layers. What can I say, experiment.

#@markdown conv0

b4c0 = False #@param {type: "boolean"}
b8c0 = False #@param {type: "boolean"}
b16c0 = False #@param {type: "boolean"}
b32c0 = False #@param {type: "boolean"}
b64c0 = False #@param {type: "boolean"}
b128c0 = False #@param {type: "boolean"}
b256c0 = False #@param {type: "boolean"}
b512c0 = False #@param {type: "boolean"}
b1024c0 = False #@param {type: "boolean"}

#@markdown conv1

b4c1 = True #@param {type: "boolean"}
b8c1 = True #@param {type: "boolean"}
b16c1 = True #@param {type: "boolean"}
b32c1 = True #@param {type: "boolean"}
b64c1 = True #@param {type: "boolean"}
b128c1 = True #@param {type: "boolean"}
b256c1 = True #@param {type: "boolean"}
b512c1 = True #@param {type: "boolean"}
b1024c1 = True #@param {type: "boolean"}

#@markdown orgb

b4torgb = True #@param {type: "boolean"}
b8torgb = True #@param {type: "boolean"}
b16torgb = True #@param {type: "boolean"}
b32torgb = True #@param {type: "boolean"}
b64torgb = True #@param {type: "boolean"}
b128torgb = True #@param {type: "boolean"}
b256torgb = True #@param {type: "boolean"}
b512torgb = True #@param {type: "boolean"}
b1024torgb = True #@param {type: "boolean"}



if b4c1:
    G._modules['synthesis'].b4.conv1 = G2._modules['synthesis'].b4.conv1      #.__dir__()
if b4torgb:
    G._modules['synthesis'].b4.torgb = G2._modules['synthesis'].b4.torgb      #.__dir__()

if b8c0:
    G._modules['synthesis'].b8.conv0 = G2._modules['synthesis'].b8.conv0      #.__dir__()
if b8c1:
    G._modules['synthesis'].b8.conv1 = G2._modules['synthesis'].b8.conv1      #.__dir__()
if b8torgb:
    G._modules['synthesis'].b8.torgb = G2._modules['synthesis'].b8.torgb      #.__dir__()

if b16c0:
    G._modules['synthesis'].b16.conv0 = G2._modules['synthesis'].b16.conv0      #.__dir__()
if b16c1:
    G._modules['synthesis'].b16.conv1 = G2._modules['synthesis'].b16.conv1      #.__dir__()
if b16torgb:
    G._modules['synthesis'].b16.torgb = G2._modules['synthesis'].b16.torgb      #.__dir__()

if b32c0:
    G._modules['synthesis'].b32.conv0 = G2._modules['synthesis'].b32.conv0      #.__dir__()
if b32c1:
    G._modules['synthesis'].b32.conv1 = G2._modules['synthesis'].b32.conv1      #.__dir__()
if b32torgb:
    G._modules['synthesis'].b32.torgb = G2._modules['synthesis'].b32.torgb      #.__dir__()

if b64c0:
    G._modules['synthesis'].b64.conv0 = G2._modules['synthesis'].b64.conv0      #.__dir__()
if b64c1:
    G._modules['synthesis'].b64.conv1 = G2._modules['synthesis'].b64.conv1      #.__dir__()
if b64torgb:
    G._modules['synthesis'].b64.torgb = G2._modules['synthesis'].b64.torgb      #.__dir__()

if b128c0:
    G._modules['synthesis'].b128.conv0 = G2._modules['synthesis'].b128.conv0      #.__dir__()
if b128c1:
    G._modules['synthesis'].b128.conv1 = G2._modules['synthesis'].b128.conv1      #.__dir__()
if b128torgb:
    G._modules['synthesis'].b128.torgb = G2._modules['synthesis'].b128.torgb      #.__dir__()

if b256c0:
    G._modules['synthesis'].b256.conv0 = G2._modules['synthesis'].b256.conv0      #.__dir__()
if b256c1:
    G._modules['synthesis'].b256.conv1 = G2._modules['synthesis'].b256.conv1      #.__dir__()
if b256torgb:
    G._modules['synthesis'].b256.torgb = G2._modules['synthesis'].b256.torgb      #.__dir__()

if b512c0:
    G._modules['synthesis'].b512.conv0 = G2._modules['synthesis'].b512.conv0      #.__dir__()
if b512c1:
    G._modules['synthesis'].b512.conv1 = G2._modules['synthesis'].b512.conv1      #.__dir__()
if b512torgb:
    G._modules['synthesis'].b512.torgb = G2._modules['synthesis'].b512.torgb      #.__dir__()

if b1024c0:
    G._modules['synthesis'].b1024.conv1 = G2._modules['synthesis'].b1024.conv1      #.__dir__()
if b1024c1:
    G._modules['synthesis'].b1024.conv0 = G2._modules['synthesis'].b1024.conv0      #.__dir__()
if b1024torgb:
    G._modules['synthesis'].b1024.torgb = G2._modules['synthesis'].b1024.torgb      #.__dir__()



# # G._modules['synthesis'].b4.conv1 = G2._modules['synthesis'].b4.conv1      #.__dir__()
# G._modules['synthesis'].b4.torgb = G2._modules['synthesis'].b4.torgb      #.__dir__()

# # G._modules['synthesis'].b8.conv0 = G2._modules['synthesis'].b8.conv0      #.__dir__()
# # G._modules['synthesis'].b8.conv1 = G2._modules['synthesis'].b8.conv1      #.__dir__()
# G._modules['synthesis'].b8.torgb = G2._modules['synthesis'].b8.torgb      #.__dir__()

# # G._modules['synthesis'].b16.conv0 = G2._modules['synthesis'].b16.conv0      #.__dir__()
# # G._modules['synthesis'].b16.conv1 = G2._modules['synthesis'].b16.conv1      #.__dir__()
# G._modules['synthesis'].b16.torgb = G2._modules['synthesis'].b16.torgb      #.__dir__()

# # G._modules['synthesis'].b32.conv0 = G2._modules['synthesis'].b32.conv0      #.__dir__()
# # G._modules['synthesis'].b32.conv1 = G2._modules['synthesis'].b32.conv1      #.__dir__()
# G._modules['synthesis'].b32.torgb = G2._modules['synthesis'].b32.torgb      #.__dir__()

# # G._modules['synthesis'].b64.conv0 = G2._modules['synthesis'].b64.conv0      #.__dir__()
# # G._modules['synthesis'].b64.conv1 = G2._modules['synthesis'].b64.conv1      #.__dir__()
# G._modules['synthesis'].b64.torgb = G2._modules['synthesis'].b64.torgb      #.__dir__()

# # G._modules['synthesis'].b128.conv0 = G2._modules['synthesis'].b128.conv0      #.__dir__()
# # G._modules['synthesis'].b128.conv1 = G2._modules['synthesis'].b128.conv1      #.__dir__()
# G._modules['synthesis'].b128.torgb = G2._modules['synthesis'].b128.torgb      #.__dir__()

# # G._modules['synthesis'].b256.conv0 = G2._modules['synthesis'].b256.conv0      #.__dir__()
# # G._modules['synthesis'].b256.conv1 = G2._modules['synthesis'].b256.conv1      #.__dir__()
# G._modules['synthesis'].b256.torgb = G2._modules['synthesis'].b256.torgb      #.__dir__()

# # G._modules['synthesis'].b512.conv0 = G2._modules['synthesis'].b512.conv0      #.__dir__()
# # G._modules['synthesis'].b512.conv1 = G2._modules['synthesis'].b512.conv1      #.__dir__()
# G._modules['synthesis'].b512.torgb = G2._modules['synthesis'].b512.torgb      #.__dir__()

# # G._modules['synthesis'].b1024.conv0 = G2._modules['synthesis'].b1024.conv0      #.__dir__()
# # G._modules['synthesis'].b1024.conv1 = G2._modules['synthesis'].b1024.conv1      #.__dir__()
# G._modules['synthesis'].b1024.torgb = G2._modules['synthesis'].b1024.torgb      #.__dir__()




imgs2 = generate_images_from_seeds(seeds, 0.7)
# display(createImageGrid(imgs2, image_size , rows))


imgs3 = imgs1 + imgs2 + imgs0

clear_output()
for _ in chunks(list(seeds), ceil(size/rows)):
  print(_)
print()
display(createImageGrid(imgs3, 0.4, rows=3))
# display(createImageGrid(imgs2, image_size , rows=1))
# createImageGrid([imgs1[1]], 1 , rows=1)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pwd

/content/stylegan2-ada-pytorch


In [ ]:
#@title # 2.1 Save new neural network


#@markdown To save new neural network type the filname (for instance ffhq+cats.pkl)
network_pkl = '/content/ffhq.pkl'
with dnnlib.util.open_url(network_pkl) as f:
    G_orig = legacy.load_network_pkl(f) # type: ignore
G_orig.keys()

G_orig['G_ema'] = G

new_network_pkl = 'ava_met' #@param {type:"string"}
download_pkl = True #@param {type:"boolean"}
with open(new_network_pkl, 'wb') as handle:
    pickle.dump(G_orig, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(new_network_pkl, 'is saved.')

if download_pkl:
    from google.colab import files
    files.download(new_network_pkl) 

generate images from pkl 

In [ ]:
!python generate.py --outdir='/content/drive/My Drive/Colab Notebooks/out2' --trunc=0.7 --seeds=8100-8125 --network='/content/drive/My Drive/Colab Notebooks/1024 pretained/ava_met.pkl'